In [5]:
from threading import Thread

import paho.mqtt.client as mqtt


class MQTT_SCU:
    def __init__(self):
        self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, message):
        print("on_message(): topic: '{}', message: '{}'".format(message.topic, message.payload.decode()))
        if message.payload.decode() == "from_charger_car_connected":
            print("Car connected from charger recieved.")
            self.stm.send('car_connected') # Send car connected to state machine
        elif message.payload.decode() == "from_charger_charge_complete":
            print("Charge complete from charger recieved.")
            self.stm.send('charging_finished') # Send charge finished to state machine
        elif message.payload.decode() == "from_charger_car_disconnected":
            print("Car disconnected from charger recieved.")
            self.stm.send('car_disconnected') # Send car disconnected to state machine
        # elif more stuff

    def start(self, broker, port):

        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("from_charger")

        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [9]:
from stmpy import Machine, Driver

import ipywidgets as widgets
from IPython.display import display

#def on_connect(rc):
#    print("Connected with result code "+str(rc))

# The callback for when a PUBLISH message is received from the server.
#def on_message(msg):
#    print(msg.topic+" "+str(msg.payload))

#myclient = MQTT_SCU()
#myclient.on_connect = on_connect
#myclient.on_message = on_message

broker, port = "test.mosquitto.org", 1883

class SCU:
    
    #def on_buttonConnect_press(self, b):
    #    self.stm.send('car_connected') # <---- here we send a signal
    
    def on_buttonCharge_press(self, b):
        self.stm.send('charge') # <---- here we send a signal
    
    #def on_buttonDisconnect_press(self, b):
    #    self.stm.send('car_disconnected') # <---- here we send a signal

    #def on_buttonChargingFinished_press(self, b):
    #    self.stm.send('charging_finished') # <---- here we send a signal
    
    def on_buttonChargingAborted_press(self, b):
        self.stm.send('charging_aborted') # <---- here we send a signal

    def on_buttonPlanWanted_press(self, b):
        self.stm.send('plan_wanted') # <---- here we send a signal
    
    def on_buttonPlanMade_press(self, b):
        self.stm.send('plan_made') # <---- here we send a signal

    def request_car_data(self):
        print('requesting car data')

    def request_charging(self):
        print('requesting charging')
    
    def request_cloud_data(self):
        print('requesting cloud data')
           
    def __init__(self):
    #    self.buttonConnect = widgets.Button(description="Connect car")
    #    self.buttonConnect.on_click(self.on_buttonConnect_press)
        
        self.buttonCharge = widgets.Button(description="Charge")
        self.buttonCharge.on_click(self.on_buttonCharge_press)

    #    self.buttonDisconnect = widgets.Button(description="Disconnect car")
    #    self.buttonDisconnect.on_click(self.on_buttonDisconnect_press)

    #    self.buttonChargingFinished = widgets.Button(description="Finish charging")
    #    self.buttonChargingFinished.on_click(self.on_buttonChargingFinished_press)

        self.buttonChargingAborted = widgets.Button(description="Abort charging")
        self.buttonChargingAborted.on_click(self.on_buttonChargingAborted_press)
        
        self.buttonPlanWanted = widgets.Button(description="request charging plan")
        self.buttonPlanWanted.on_click(self.on_buttonPlanWanted_press)

        self.buttonPlanMade = widgets.Button(description="plan is made")
        self.buttonPlanMade.on_click(self.on_buttonPlanMade_press)

        display(self.buttonCharge, self.buttonChargingAborted, self.buttonPlanWanted)#, self.buttonPlanMadeself.buttonDisconnect, self.buttonChargingFinished, self.buttonConnect)


scu = SCU()

t0_initial = {'source': 'initial',
      'target': 'idle'}

t1_car_connected = {'trigger': 'car_connected',
        'source': 'idle',
        'target': 'carConnected'
}

t2_plan_wanted = {'trigger': 'plan_wanted',
        'source': 'carConnected',
        'target': 'createPlan'
}
t3_plan_made = {'trigger': 'plan_made',
        'source': 'createPlan',
        'target': 'carConnected'
}
t4_charge = {'trigger': 'charge',
        'source': 'carConnected',
        'target': 'chargeCar'
}
t5_charging_finished = {'trigger': 'charging_finished',
        'source': 'chargeCar',
        'target': 'carConnected'
}
t6_charging_aborted = {'trigger': 'charging_aborted',
        'source': 'chargeCar',
        'target': 'carConnected'
}
t7_car_disconnected = {'trigger': 'car_disconnected',
        'source': 'carConnected',
        'target': 'idle'
}

idle = {'name': 'idle'}

carConnected = {'name': 'carConnected',
                'entry': 'request_car_data'}

chargeCar = {'name': 'chargeCar',
             'entry': 'request_charging'}

createPlan = {'name': 'createPlan',
              'entry': 'request_cloud_data'}

machine = Machine(name='SCU', transitions=[t0_initial, t1_car_connected, t2_plan_wanted, t3_plan_made, t4_charge, t5_charging_finished, t6_charging_aborted, t7_car_disconnected],
                                states=[idle, carConnected, chargeCar, createPlan], obj=scu)

scu.stm = machine

driver = Driver()
driver.add_machine(machine)

mqtt_scu = MQTT_SCU()
scu.mqtt_client = mqtt_scu.client
mqtt_scu.stm_driver = driver

driver.start()
mqtt_scu.start(broker, port)

Button(description='Charge', style=ButtonStyle())

Button(description='Abort charging', style=ButtonStyle())

Button(description='request charging plan', style=ButtonStyle())

Connecting to test.mosquitto.org:1883


C:\Users\Andre\AppData\Local\Temp\ipykernel_7040\2003887535.py:8: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)


Exception in thread Thread-44:
Traceback (most recent call last):
  File "c:\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\Python39\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Python39\lib\site-packages\paho\mqtt\client.py", line 2291, in loop_forever
    rc = self._loop(timeout)
  File "c:\Python39\lib\site-packages\paho\mqtt\client.py", line 1680, in _loop
    rc = self.loop_read()
  File "c:\Python39\lib\site-packages\paho\mqtt\client.py", line 2094, in loop_read
    rc = self._packet_read()
  File "c:\Python39\lib\site-packages\paho\mqtt\client.py", line 3137, in _packet_read
    rc = self._packet_handle()
  File "c:\Python39\lib\site-packages\paho\mqtt\client.py", line 3809, in _packet_handle
    return self._handle_connack()
  File "c:\Python39\lib\site-packages\paho\mqtt\clien